# 单历元定位要求基于GPS系统的C1C和C2W观测值，组成无电离层组合的观测值，计算2019年7月15号06:55:30的坐标。
## hkws1960.19o是观测文件，igs20621.sp3是精密星历文件。

##### 1.读取hkws1960.19o观测文件，将GPS系统的C1C和C2W观测值分别存在两个列表中。

In [1]:
import numpy as np

obsfile = 'hkws1960.19o'
satnames = [] # 保存卫星编号
c1c = [] # 保存L1伪距观测值
c2w = [] # 保存L2伪距观测值

with open(obsfile) as file_object:
    for line in file_object:
        if line[2:29] == '2019 07 15 06 55 30.0000000':
            number = int(line[32:35])
            for i in range(number):
                if line[0] == 'G':  # 选出所有GPS观测数据
                    # 每个数据占16列，第i个数据的起始列数为4 + 16(𝑖 − 1)
                    satnames.append(line[0:3])
                    c1c.append(float(line[3+16*(1-1):4+16*(1-1)+16]))  # C1C为第1个数据
                    c2w.append(float(line[3+16*(9-1)-1:4+16*(9-1)+16]))  #C2W为第9个数据
                line = file_object.readline() # 读取行不以字符'G'开头，继续向下读取
            break # 收集完'2019 07 15 06 55 30.0000000'时的数据即可退出文件读取循环

if len(c1c)==len(c2w):
    satnum = len(c1c) 
    
p1 = np.array(c1c) #存在1*9的矩阵中
p2 = np.array(c2w)

In [2]:
satnames

['G02', 'G05', 'G06', 'G09', 'G12', 'G17', 'G19', 'G23', 'G28']

##### 2.读取igs20621.sp3精密星历文件，获得卫星所在星历的时间、位置和钟差信息


In [3]:
igsT = [] # 存储观测卫星在7月15各个所取星历的时间 (min)
sXdata = [] # 存储观测卫星在各个所取星历时间的x坐标 (km)
sYdata = [] # 存储观测卫星在各个所取星历时间的y坐标 (km)
sZdata = [] # 存储观测卫星在各个所取星历时间的z坐标 (km)
sClock = [] # 存储观测卫星在各个所取星历时间的钟差(μs)
    
def read_igs(time):
    igsT.clear()
    sXdata.clear()
    sYdata.clear()
    sZdata.clear()
    sClock.clear()
  
    hour = time // 60
    minute = (time-hour*60) % 60 // 1 
    count = minute // 15 
    start_time = hour * 60 + count * 15 - 4*15   # 0 -45 -30 -15 0 15 30 45 找到起始取点时间
    
    with open('igs20621.sp3') as file_object:
        for line in file_object:
            if line[0] == '*' and float(line[14:16]) == (start_time // 60) and float(line[17:19]) == (start_time % 60):
                igsT.append(float(line[14:16])*60 + float(line[17:19]) + float(line[20:31])/60) 
                # 卫星位置（km）和钟差（μs）依次为X、Y、Z、钟差每个数据占14列
                # 第i个数据的起始列为 5 + 14(i - 1)
                while(len(igsT)<=10):   # 以时间t为准，向前找5组精密星历，向后找5组精密星历，需10组
                    line = file_object.readline()
                    for n in range(satnum):  # 读一组选取时间点的星历，存储9个观测卫星参数
                        while line[1:4] != satnames[n]:
                            line = file_object.readline()
                        sXdata.append(float(line[4+14*(1-1):5+14*(1-1)+14]))                            
                        sYdata.append(float(line[4+14*(2-1):5+14*(2-1)+14]))
                        sZdata.append(float(line[4+14*(3-1):5+14*(3-1)+14]))
                        sClock.append(float(line[4+14*(4-1):5+14*(4-1)+14]))
                    if len(igsT) == 10:
                        break
                    else:
                        while(line[0] != '*'):
                            line = file_object.readline()
                        igsT.append(float(line[14:16])*60 + float(line[17:19]) + float(line[20:31])/60) 
            else:
                line = file_object.readline()
            
    sXmtr = np.array(sXdata).reshape(10, satnum)
    sYmtr = np.array(sYdata).reshape(10, satnum)
    sZmtr = np.array(sZdata).reshape(10, satnum)
    sCmtr = np.array(sClock).reshape(10, satnum)
    
    return(sXmtr, sYmtr, sZmtr, sCmtr)
    


In [4]:
5*60+45

345

In [5]:
time = 6*60 + 55 + 30/60
hour = time // 60
minute = (time-hour*60) % 60 // 1 
count = minute // 15
start_time = hour * 60 + count * 15 - 4*15  # 0 -45 -30 -15 0 15 30 45 找到起始取点时间

In [6]:
start_time // 60

5.0

##### 3.以igsT时间为自变量使用9阶拉格朗日插值，解算出所求时刻的观测卫星信息

In [7]:
from scipy.interpolate import lagrange

def interpolation(t): 
    # 对于每个时间，重新读取接近该时间点的10个星历
    sXmtr, sYmtr, sZmtr, sCmtr = read_igs(t)
    xs = []
    ys = []
    zs = []
    dts = []
    # 对于9个卫星，分别使用10个历元的卫星位置和钟差插值时间t
    # 得出每个卫星的位置在时间t的位置和钟差
    for i in range(satnum):
        sx_poly = lagrange(igsT, sXmtr[:,i])
        xs.append(sx_poly(t))
        sy_poly = lagrange(igsT, sYmtr[:,i])
        ys.append(sy_poly(t))
        sz_poly = lagrange(igsT, sZmtr[:,i])
        zs.append(sz_poly(t))
        sc_poly = lagrange(igsT, sCmtr[:,i])
        dts.append(sc_poly(t))

    # 将观测卫星的位置和钟差转换为矩阵
    satPos = np.array([xs, ys, zs])
    dTs = np.array(dts)
    satPos *= 1000 # km转换为m
    dTs *= 10**(-6) # μs转换为s
    return (satPos, dTs)


##### 4.使用最小二乘法求接收机x,y,z和dx,dy,dz,ddt。组成观测方程(V=L-A*dX, P)，P可取单位阵。


In [8]:
from math import sqrt
from numpy.linalg import inv

f1 = 1575.42 # C1C载波频率
f2 = 1227.6 # C2W载波频率
c = 299792458
PC = f1**2/(f1**2 - f2**2)*p1 - f2**2/(f1**2 - f2**2)*p2 # 无电离层组合观测值
dX = np.ones(4)
X = np.zeros(4)
t0 = 6*60 + 55 + 30/60 
sP, dT =  interpolation(t0)

while sqrt(dX[0]**2+dX[1]**2+dX[2]**2) >= 0.0001:
    Li = []
    Ai = []
    for i in range(satnum):
        rho = sqrt((sP[0,i]-X[0])**2+(sP[1,i]-X[1])**2+(sP[2,i]-X[2])**2)
        t = t0 - rho / c / 60
        sP, dT = interpolation(t)  # 更新时间，重新插值
        Li.append(PC[i]-rho-c*X[3]+c*dT[i])
        Ai.append([(X[0]-sP[0,i])/rho, (X[1]-sP[1,i])/rho, (X[2]-sP[2,i])/rho, c])
    
    A = np.array(Ai).reshape(satnum, 4)
    L = np.array(Li)
    Q = inv(A.T.dot(A))
    dX = Q.dot(A.T.dot(L))
    X = X + dX
    


In [9]:
X

array([-2.42990871e+06,  5.37378683e+06,  2.41907242e+06, -7.29361443e-07])

In [10]:
dX

array([-1.59410745e-07,  3.13743336e-06,  1.03115627e-06,  4.70443994e-15])